In [1]:
import json
import pandas as pd

In [2]:
import requests,gzip,json

In [3]:
gzfileUrl="https://pm25.lass-net.org/data/last-all-airbox.json.gz"
gzflieSaveName="last-all-airbox.json.gz"
r = requests.get(gzfileUrl)
open(gzflieSaveName, 'wb').write(r.content)

f = gzip.open(gzflieSaveName,'r')
jdata = f.read()
f.close()
data = json.loads(jdata)
print(data)

{'c_d0_source': 'AS-IISNRL', 'num_of_records': 3099, 'source': 'last-all-airbox by IIS-NRL', 'version': '2020-05-28T08:36:38Z', 'descriptions': {'URL': 'https://pm25.lass-net.org/data/description.json', 'c_d0_method': 'method/days/distance(km)', 'c_d0': 'calibration PM2.5 (ug/m3)'}, 'feeds': [{'hcho': '0', 'gps_num': 9.0, 'app': 'AirBox', 'gps_alt': 2.0, 's_d2': 22.0, 'odm': 'acelink.edimax', 's_d0': 38.0, 's_d1': 46.0, 's_h0': 100.0, 'SiteName': '雲林縣縣立僑真國小', 'gps_fix': 1.0, 'gps_lat': 23.6676036, 's_t0': 29.6, 'timestamp': '2020-05-28T08:30:05Z', 'adf_status': '0', 'gps_lon': 120.4744785, 'SiteAddr': '雲林縣斗南鎮延平路二段六四二號', 'date': '2020-05-28', 'device_id': '08BEAC0A0270', 'c_d0': 65.84, 'fw_ver': 'v1.06', 's_g1': 0.0, 'time': '08:30:05', 'model': 'AI-1003WN', 'c_d0_method': 'BRR/21/8.7'}, {'hcho': '0', 'gps_num': 9.0, 'app': 'AirBox', 'gps_alt': 2.0, 's_d2': 8.0, 'odm': 'acelink.edimax', 's_d0': 13.0, 's_d1': 14.0, 's_h0': 98.0, 'SiteName': '嘉義縣縣立大崙國小', 'gps_fix': 1.0, 'gps_lat': 23.4437

In [4]:
def get_target_value(key, dic, tmp_list):
    """
    :param key: 目标key值
    :param dic: JSON数据
    :param tmp_list: 用于存储获取的数据
    :return: list
    """
    if not isinstance(dic, dict) or not isinstance(tmp_list, list):  # 对传入数据进行格式校验
        return 'argv[1] not an dict or argv[-1] not an list '

    if key in dic.keys():
        tmp_list.append(dic[key])  # 传入数据存在则存入tmp_list

    for value in dic.values():  # 传入数据不符合则对其value值进行遍历
        if isinstance(value, dict):
            get_target_value(key, value, tmp_list)  # 传入数据的value值是字典，则直接调用自身
        elif isinstance(value, (list, tuple)):
            _get_value(key, value, tmp_list)  # 传入数据的value值是列表或者元组，则调用_get_value


    return tmp_list

def _get_value(key, val, tmp_list):
    for val_ in val:
        if isinstance(val_, dict):  
            get_target_value(key, val_, tmp_list)  # 传入数据的value值是字典，则调用get_target_value
        elif isinstance(val_, (list, tuple)):
            _get_value(key, val_, tmp_list)   # 传入数据的value值是列表或者元组，则调用自身


In [5]:
site_all = get_target_value('SiteName', data, [])
site_id_all = get_target_value('device_id', data, [])

In [6]:
df = pd.DataFrame(site_all, site_id_all)

In [7]:
df['device_id'] = site_id_all

In [8]:
columns_name = ['SiteName', 'device_id']

In [9]:
df.columns = columns_name

In [10]:
df.reset_index()


,index,SiteName,device_id
0,08BEAC0A0270,雲林縣縣立僑真國小,08BEAC0A0270
1,08BEAC0A06D8,嘉義縣縣立大崙國小,08BEAC0A06D8
2,08BEAC0A0274,臺中市立居仁國中,08BEAC0A0274
3,08BEAC0A0276,雲林縣縣立大東國小,08BEAC0A0276
4,08BEAC0A06D4,臺南市市立新營區新興國小,08BEAC0A06D4
5,08BEAC0A0278,桃園市市立雙龍國小,08BEAC0A0278
6,08BEAC0A06D6,嘉義縣縣立竹崎高中附設國中,08BEAC0A06D6
7,08BEAC0A06D0,桃園市市立霞雲國小,08BEAC0A06D0
8,08BEAC0A06D2,臺南市市立德南國小,08BEAC0A06D2
9,74DA38F7C368,新北市秀朗國小,74DA38F7C368


In [11]:
df.set_index('SiteName')

,device_id
SiteName,
雲林縣縣立僑真國小,08BEAC0A0270
嘉義縣縣立大崙國小,08BEAC0A06D8
臺中市立居仁國中,08BEAC0A0274
雲林縣縣立大東國小,08BEAC0A0276
臺南市市立新營區新興國小,08BEAC0A06D4
桃園市市立雙龍國小,08BEAC0A0278
嘉義縣縣立竹崎高中附設國中,08BEAC0A06D6
桃園市市立霞雲國小,08BEAC0A06D0
臺南市市立德南國小,08BEAC0A06D2
